In [ ]:
# execute as needed
# !pip install -r requirements.txt

## Process Books

In [1]:
import sys

sys.path.append("../")

RAW_BOOKS_FOLDER_PATH = "../raw_data/books/"
PROCESSED_BOOKS_FOLDER_PATH = "../processed_data/books/"

In [3]:
import os
import PyPDF2
from modules.embedding_tracking import add_new_file, file_exists

available_files = os.listdir(RAW_BOOKS_FOLDER_PATH)

for i in available_files:
    print(f"Processing {i}")
    
    processed_fname = ".".join(i.split(".")[:-1])+".txt"
    print(processed_fname)
    
    if file_exists(processed_fname):
        print(f"Skipping {i} as it has already been processed.")
        continue
    
    text = ""
    if i.endswith(".pdf"):
        fname = os.path.join(RAW_BOOKS_FOLDER_PATH, i)
        reader = PyPDF2.PdfReader(fname)
        for page_num in range(len(reader.pages)):
            page = reader.pages[page_num]
            text += page.extract_text()
            
    else:
        # invalid file types
        print(f"Skipping {i} as it is not a valid file.")
        continue
    
    with open(os.path.join(PROCESSED_BOOKS_FOLDER_PATH, processed_fname), "w", encoding="utf-8") as f:
        f.write(text)
        add_new_file(processed_fname)
        
    print(f"Finished processing {i}.")

Processing Sunrise on the Reaping.pdf
Sunrise on the Reaping.txt
Skipping Sunrise on the Reaping.pdf as it has already been processed.
Processing All Fours (Miranda July).pdf
All Fours (Miranda July).txt
Skipping All Fours (Miranda July).pdf as it has already been processed.


### Store to storage as vectors

In [4]:
from modules import accessor
from modules import embedding_tracking

filenames = os.listdir(PROCESSED_BOOKS_FOLDER_PATH)
embedding_functions = list(accessor.EMBEDDING_FUNCTIONS.keys())

for filename in filenames:
    with open(os.path.join(PROCESSED_BOOKS_FOLDER_PATH, filename), "r", encoding="utf-8") as file:
        text = file.read()
        print(f"Tokenizing {filename}")
        
        for embedding_function in embedding_functions:
            suffix = "full_text"
            if not embedding_tracking.is_file_processed(filename, "{}_{}".format(embedding_function, suffix)):
                try:
                    ids = accessor.insert(
                        data=[text],
                        embedding_func=embedding_function,
                        custom_suffix=suffix,
                    )
                    embedding_tracking.mark_file_processed(filename, suffix)
                except Exception as e:
                    print(f"Error storing {suffix} for {filename}: {e}")
            else:
                print(f"Skipping {filename} for {embedding_function} as it has already been processed with suffix {suffix}.")
                    
            suffix = "512_chunks"
            if not embedding_tracking.is_file_processed(filename, "{}_{}".format(embedding_function, suffix)):
                chunked_texts = [text[i:i+512] for i in range(0, len(text), 512)]
                try:
                    ids = accessor.insert(
                        data=chunked_texts,
                        embedding_func=embedding_function,
                        custom_suffix=suffix,
                    )
                    embedding_tracking.mark_file_processed(filename, suffix)
                except Exception as e:
                    print(f"Error storing {suffix} for {filename}: {e}")
            else:
                print(f"Skipping {filename} for {embedding_function} as it has already been processed with suffix {suffix}.")
            
    print(f"Finished processing {filename} for embeddings.")

Tokenizing All Fours (Miranda July).txt
Inserted data with IDs: ['7a9cb11f-8fa7-4a5d-8c1d-1e9276c08ac0'] into collection 'nomic-embed-text_collection_full_text'.
Inserted data with IDs: ['661d27f1-4cec-4c75-a42e-708e88a0eadb', 'a577d963-b27e-4e67-b7d9-84a1cf47be9b', '7dacac90-e67a-4264-829b-b55ac2d4fe8a', '70158f3e-3d9e-4034-b77a-de8c370f7d08', '2a778f8c-c3a9-4153-9013-b3d4f3c3a95f', '1923c047-3cc9-4e36-ab67-b9ffb53b42fd', '5353cc4a-84e8-40b2-a4f0-6ff0d391966e', '5643055b-2d0a-4f11-8df5-8ef741f5de40', 'f325665c-4a13-47d9-b583-d869ab677590', 'ed3e12fb-859c-47a0-8e5f-5f0c762e617a', 'aa39ca82-8266-471c-b00e-19804515aae7', '173be6e3-b7bd-4a16-a4d2-b4392a135553', '197cb251-1c25-4bdf-9635-bce52b0f6acc', 'bf2bc656-2f23-4be2-8b64-10ba1599baf4', '3ff11398-bcd4-49dd-8506-85e24c440db8', '25a31a3b-2859-483b-8022-0687378de5c7', 'bf2214f2-2a4c-49fa-b9d2-56d9671b459f', 'f43ff5d7-01fd-4c03-90f0-c1a0bec3f06f', 'b53955a2-97b6-454b-a9f9-593028571b8c', '48b0960a-e496-41fd-9965-0824a6eba1cd', 'ad30dc76-82c

In [7]:
available_collections = accessor.list_collections()
print(f"Available collections: {available_collections}")

documents = accessor.get(
    query="What book has the word Sunrise in it?",
    embedding_func="all-minilm",
    custom_suffix="512_chunks",
    n_results=10
)
for i in documents:
    print(i.page_content[:100])
    print("-------------------------")

Available collections: ['nomic-embed-text_collection_full_text', 'mxbai-embed-large_collection_512_chunks', 'all-minilm_collection_512_chunks', 'bge-m3_collection_512_chunks', 'snowflake-arctic-embed_collection_512_chunks', 'nomic-embed-text_collection_512_chunks', 'bge-m3_collection_full_text', 'snowflake-arctic-embed2_collection_full_text', 'snowflake-arctic-embed_collection_full_text', 'mxbai-embed-large_collection_full_text', 'snowflake-arctic-embed2_collection_512_chunks', 'langchain', 'all-minilm_collection_full_text']
eps the District 12 of fice in shape — thanks for the super job you do
making sure everything runs s
-------------------------
or design adapted for ebook by Cora W igen
This is a work of fiction. Names, characters, places, and
-------------------------
 is something at my window lattice;
Let me see, then, what ther eat is, and this mystery explor e —

-------------------------
into
focus. They’re not lined with instruments of pain but towering shelves of
books. Th

## Process Movie Transcripts